# Solving TSP problem and drowing the optimal path on the map.

In [2]:
!pip install python-tsp

In [4]:
import numpy as np
import json
import math
import random
import time
import cv2
from python_tsp.heuristics import solve_tsp_simulated_annealing
from python_tsp.heuristics import solve_tsp_local_search

def draw_circles(img, x, y):
    center = (x, y)
    radius = 40
    thickness = 5
    img = cv2.circle(img, center, radius, (0, 0, 255), thickness)
    radius = 4
    thickness = -1
    img = cv2.circle(img, center, radius, (0, 0, 255), thickness)
    return img

def draw_line(img, x1, y1, x2, y2):
    cos = abs(x1 - x2) / math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    sin = math.sqrt(1 - cos*cos)
    r = 40

    if (x1 > x2):
        x2 += (int)(r*cos)
        x1 -= (int)(r*cos)
    else:
        x2 -= (int)(r*cos)
        x1 += (int)(r*cos)

    if (y1 > y2):
        y2 += (int)(r*sin)
        y1 -= (int)(r*sin)
    else:
        y2 -= (int)(r*sin)
        y1 += (int)(r*sin)

    p1 = (x1, y1)
    p2 = (x2, y2)
    color = (0, 0, 255)
    thickness = 5
    img = cv2.line(img, p1, p2, color, thickness)
    return img


day = "4" # номер дня цикла
sets = np.array([0, 0, 0, 1])
setnames = np.array(["cups", "pentacles", "swords", "wands"])
Ncards = 0 # количество карт
for i in range(4):
    if sets[i] == 1: Ncards += 14
dist = np.eye(Ncards + 1) # матрица расстояний

scale = 59.2 # коэффициен масштабирования для отрисовки

# сдвиги для отрисовки
xShifts = [[1976, 1380, 852, 828, 1246, 1905],    # cups
           [968, 2047, 1288, 4537, 885, 1282],    # pentacles
           [1272, 973, 1255, 4905, 4347, 847],    # swords
           [3310, 1190, 2818, 1333, 1560, 4536]]  # wands

yShifts = [[3162, 2458, 2598, 2759, 2921, 3203],  # cups
           [2725, 2217, 3004, 3825, 2648, 2966],  # pentacles
           [2940, 2212, 3447, 3570, 3548, 2697],  # swords
           [3010, 2290, 2886, 2490, 2430, 3540]]  # wands

# получаем координаты карт
with open("/content/drive/MyDrive/Colab Notebooks/RDRII files/data_file.json", "r") as read_file:
    data = json.load(read_file)

# строим массивы координат
x = np.zeros(Ncards + 1)
y = np.zeros(Ncards + 1)
tmp = np.zeros(14)

count = 0
for i in range(4):
    if sets[i] == 1:
        for j in range(14):
            x[14*count + j + 1] = data[setnames[i]][day][j]["lat"] + 120
            y[14*count + j + 1] = data[setnames[i]][day][j]["lng"]
        for j in range(14):
            tmp[j] = x[14*count + j + 1]
            x[14*count + j + 1] = y[14*count + j + 1]
            y[14*count + j + 1] = tmp[j]
        count += 1

# строим матрицу расстояний
for i in range(1, Ncards + 1):
    for j in range(1, i):
        a = math.sqrt((x[i] - x[j])**2 + (y[i] - y[j])**2)
        dist[i][j] = a
        dist[j][i] = a

# решение 1 ----------------------------------------------------------------------------------------------------------------------
print("simulated annealing")
start_time = time.time()
permutation, distance = solve_tsp_simulated_annealing(dist)
end_time = time.time()
print(distance)
print(permutation)
print("--- %s seconds ---" % (end_time - start_time))

# загрузка карты
image = cv2.imread("/content/drive/MyDrive/Colab Notebooks/RDRII files/rdr_map.jpg")
height, width = image.shape[:2]

# подготовка координат для отрисовки решения на карте
count = 0
for i in range(4):
    if sets[i] == 1:
        minx = min(x[14*count + 1:14*count + 14 + 1])
        miny = min(y[14*count + 1:14*count + 14 + 1])
        for j in range(14):
            x[14*count + j + 1] -= minx
            y[14*count + j + 1] -= miny
        count += 1

count = 0
for i in range(4):
    if sets[i] == 1:
        for j in range(14):
            x[14*count + j + 1] = (int)(scale*x[14*count + j + 1]) + xShifts[i][int(day) - 1]
            y[14*count + j + 1] = height - (int)(scale*y[14*count + j + 1]) - yShifts[i][int(day) - 1]
        count += 1

# создание имени выходного файла
filename = ""
count = 0
for i in range(4):
    if sets[i] == 1:
        if count > 0:
            filename += "_"
        filename += setnames[i]
        count += 1
filename += "_day"
filename += day

# отрисовка решения
for i in range(1, Ncards + 1):
    image = draw_circles(image, (int)(x[permutation[i]]), (int)(y[permutation[i]]))

for i in range(1, Ncards):
    image = draw_line(image, (int)(x[permutation[i]]), (int)(y[permutation[i]]), (int)(x[permutation[i+1]]), (int)(y[permutation[i+1]]))

'''
scale_percent = 80
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dsize = (width, height)
image = cv2.resize(image, dsize)
'''

cv2.imwrite("/content/drive/MyDrive/Colab Notebooks/RDRII files/" + filename + ".jpg", image)
#cv2.imwrite(filename + "_simulated_annealing.jpg", image)
print("")

'''
# решение 2 ----------------------------------------------------------------------------------------------------------------------
initpermutation = list()
initpermutation.append(0)

numb = list()
for j in range(1, Ncards + 1):
    numb.append(j)
for j in range(Ncards):
    a = random.choice(numb)
    initpermutation.append(a)
    numb.remove(a)

print("local search")
start_time = time.time()
permutation, distance = solve_tsp_local_search(dist, x0=initpermutation)
end_time = time.time()
print(distance)
print(permutation)
print("--- %s seconds ---" % (end_time - start_time))

# загрузка карты
image = cv2.imread('rdr_map.jpg')
height, width = image.shape[:2]

# отрисовка решения
for i in range(1, Ncards + 1):
    image = draw_circles(image, (int)(x[permutation[i]]), (int)(y[permutation[i]]))

for i in range(1, Ncards):
    image = draw_line(image, (int)(x[permutation[i]]), (int)(y[permutation[i]]), (int)(x[permutation[i+1]]), (int)(y[permutation[i+1]]))
'''
'''
scale_percent = 80
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dsize = (width, height)
image = cv2.resize(image, dsize)
'''
'''
cv2.imwrite(filename + "_local_search.jpg", image)
print("")
'''

print(filename)
print("")

simulated annealing
272.7269760856223
[0, 11, 2, 10, 6, 5, 8, 7, 1, 12, 4, 13, 3, 9, 14]
--- 0.37831568717956543 seconds ---

wands_day4



# Telegram bot

In [ ]:
# -*- coding: cp1251 -*-
from os import waitpid
import telebot;
from telebot import types
import datetime

bot = telebot.TeleBot('5348879862:AAHetIidhaH3iKbiYN2xE0xzICWpRZL1ITg');

cups = list()
pentacles = list()
swords = list()
wands = list()

userid = list()

@bot.message_handler(commands=['start'])    
def Start(message):
    global cups
    global pentacles
    global swords
    global wands
    global userid
    if message.from_user.id not in userid:
        userid.append(message.from_user.id)
        cups.append(0)
        pentacles.append(0)
        swords.append(0)
        wands.append(0)
        s = "New user was added.   User: " + "(" + str(message.from_user.id) + ")"
        if (message.from_user.first_name != None):
            s += " "
            s += message.from_user.first_name
        if (message.from_user.last_name != None):
            s += " "
            s += message.from_user.last_name
        if (message.from_user.username != None):
            s += " ("
            s += message.from_user.username
            s += ")"
        print(s)
        
    bot.send_message(message.from_user.id, "This bot shows you the best routes to collect tarot cards in Red Dead Online")
    
    AskAboutSets(message)


@bot.message_handler(content_types=['text'])
def Reply(message):
    global cups
    global pentacles
    global swords
    global wands
    
    if message.text == "Сhoose another cards":
            AskAboutSets(message)
    
    if message.from_user.id not in userid:
        bot.send_message(message.from_user.id, "Unfortunately we have to start over")
        AskAboutSets(message)
    else: 
        if message.text == "Add Cups":
            cups[userid.index(message.from_user.id)] = 1
            ChooseSets(message)
        if message.text == "Exclude Cups":
            cups[userid.index(message.from_user.id)] = 0
            ChooseSets(message)

        if message.text == "Add Pentacles":
            pentacles[userid.index(message.from_user.id)] = 1
            ChooseSets(message)
        if message.text == "Exclude Pentacles":
            pentacles[userid.index(message.from_user.id)] = 0
            ChooseSets(message)

        if message.text == "Add Swords":
            swords[userid.index(message.from_user.id)] = 1
            ChooseSets(message)
        if message.text == "Exclude Swords":
            swords[userid.index(message.from_user.id)] = 0
            ChooseSets(message)

        if message.text == "Add Wands":
            wands[userid.index(message.from_user.id)] = 1
            ChooseSets(message)
        if message.text == "Exclude Wands":
            wands[userid.index(message.from_user.id)] = 0
            ChooseSets(message)

        if message.text == "Next":
            ChooseDay(message)
        if message.text == "Day 1":
            Answer(message, 1)
        if message.text == "Day 2":
            Answer(message, 2)
        if message.text == "Day 3":
            Answer(message, 3)
        if message.text == "Day 4":
            Answer(message, 4)
        if message.text == "Day 5":
            Answer(message, 5)
        if message.text == "Day 6":
            Answer(message, 6)
        
        if message.text == "Choose anoher day":
            ChooseDay(message)

def AskAboutSets(message):
    global cups
    global pentacles
    global swords
    global wands
    global userid
    
    if message.from_user.id not in userid:
        userid.append(message.from_user.id)
        cups.append(0)
        pentacles.append(0)
        swords.append(0)
        wands.append(0)
 
        s = "New user was added.   User: (" + str(message.from_user.id) + ")"
        if (message.from_user.first_name != None):
            s += " "
            s += message.from_user.first_name
        if (message.from_user.last_name != None):
            s += " "
            s += message.from_user.last_name
        if (message.from_user.username != None):
            s += " ("
            s += message.from_user.username
            s += ")"
        print(s)

    cups[userid.index(message.from_user.id)] = 0
    pentacles[userid.index(message.from_user.id)] = 0
    swords[userid.index(message.from_user.id)] = 0
    wands[userid.index(message.from_user.id)] = 0

    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    
    key_cups = types.KeyboardButton("Add Cups")
    key_pentacles = types.KeyboardButton("Add Pentacles")
    key_swords = types.KeyboardButton("Add Swords")
    key_wands = types.KeyboardButton("Add Wands")
    keyboard.add(key_cups, key_pentacles, key_swords, key_wands)

    question = "What cards you want to collect?";
    bot.send_message(message.from_user.id, text=question, reply_markup=keyboard)


def ChooseSets(message):
    global cups
    global pentacles
    global swords
    global wands
    global userid

    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    
    if cups[userid.index(message.from_user.id)] == 1:
        bot.send_message(message.from_user.id, text="Cups included")
        key_cups = types.KeyboardButton("Exclude Cups")
    else:
        key_cups = types.KeyboardButton("Add Cups")

    if pentacles[userid.index(message.from_user.id)] == 1:
        bot.send_message(message.from_user.id, text="Pentacles included")
        key_pentacles = types.KeyboardButton("Exclude Pentacles")
    else:
        key_pentacles = types.KeyboardButton("Add Pentacles")

    if swords[userid.index(message.from_user.id)] == 1:
        bot.send_message(message.from_user.id, text="Swords included")
        key_swords = types.KeyboardButton("Exclude Swords")
    else:
        key_swords = types.KeyboardButton("Add Swords")

    if wands[userid.index(message.from_user.id)] == 1:
        bot.send_message(message.from_user.id, text="Wands included")
        key_wands = types.KeyboardButton("Exclude Wands")
    else:
        key_wands = types.KeyboardButton("Add Wands")

    if (cups[userid.index(message.from_user.id)] == 1 or pentacles[userid.index(message.from_user.id)] == 1 or swords[userid.index(message.from_user.id)] == 1 or wands[userid.index(message.from_user.id)] == 1):
        key_next = types.KeyboardButton("Next")
        keyboard.add(key_cups, key_pentacles, key_swords, key_wands, key_next)
    else:
        keyboard.add(key_cups, key_pentacles, key_swords, key_wands)

    question = "Something else?";
    bot.send_message(message.from_user.id, text=question, reply_markup=keyboard)

def ChooseDay(message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    
    key_day1 = types.KeyboardButton("Day 1")
    key_day2 = types.KeyboardButton("Day 2")
    key_day3 = types.KeyboardButton("Day 3")
    key_day4 = types.KeyboardButton("Day 4")
    key_day5 = types.KeyboardButton("Day 5")
    key_day6 = types.KeyboardButton("Day 6")
    keyboard.add(key_day1, key_day2, key_day3, key_day4, key_day5, key_day6)

    bot.send_message(message.from_user.id, text="Choose day", reply_markup=keyboard)

def Answer(message, day):
    global cups
    global pentacles
    global swords
    global wands
    have_image = 0
    count = 0
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True);

    key_wait = types.KeyboardButton("I'm waiting...")
    keyboard.add(key_wait)
    bot.send_message(message.from_user.id, text="Give me a sec...", reply_markup=keyboard)

    filename = "C:\RDRII_bot\ "
    filename = filename[:-1]

    if cups[userid.index(message.from_user.id)] == 1:
        filename += "cups"
        count += 1
    if pentacles[userid.index(message.from_user.id)] == 1:
        if count > 0:
            filename += "_"
        filename += "pentacles"
        count += 1
    if swords[userid.index(message.from_user.id)] == 1:
        if count > 0:
            filename += "_"
        filename += "swords"
        count += 1
    if wands[userid.index(message.from_user.id)] == 1:
        if count > 0:
            filename += "_"
        filename += "wands"
        count += 1

    filename = filename + "_day" + str(day) + ".jpg"
    today = datetime.datetime.today()
    s = "(" + str(message.from_user.id) + ")"
    if (message.from_user.first_name != None):
        s += " "
        s += message.from_user.first_name
    if (message.from_user.last_name != None):
        s += " "
        s += message.from_user.last_name
    if (message.from_user.username != None):
        s += " ("
        s += message.from_user.username
        s += ")"
    s += " want "
    s += filename
    s += "     "
    s += today.strftime("%d-%m-%Y  %H:%M:%S")
    print(s)
    
    try:
        image  =  open(filename, 'rb')
        bot.send_document(message.from_user.id, image)
        have_image = 1
        today = datetime.datetime.today()
        s = "(" + str(message.from_user.id) + ")"
        if (message.from_user.first_name != None):
            s += " "
            s += message.from_user.first_name
        if (message.from_user.last_name != None):
            s += " "
            s += message.from_user.last_name
        if (message.from_user.username != None):
            s += " ("
            s += message.from_user.username
            s += ")"
        s += " receive "
        s += filename
        s += "     "
        s += today.strftime("%d-%m-%Y  %H:%M:%S")
        print(s)
    except Exception:
        have_image = 0
        today = datetime.datetime.today()
        s = "(" + str(message.from_user.id) + ")"
        if (message.from_user.first_name != None):
            s += " "
            s += message.from_user.first_name
        if (message.from_user.last_name != None):
            s += " "
            s += message.from_user.last_name
        if (message.from_user.username != None):
            s += " ("
            s += message.from_user.username
            s += ")"
        s += " did not receive "
        s += filename
        s += "     "
        s += today.strftime("%d-%m-%Y  %H:%M:%S")
        print(s)

    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    
    key_collect = types.KeyboardButton("Сhoose another cards")
    key_choose_day = types.KeyboardButton("Choose anoher day")
    keyboard.add(key_collect, key_choose_day)

    if have_image == 1:
        bot.send_message(message.from_user.id, "Collect it!", reply_markup=keyboard)
    else:
        bot.send_message(message.from_user.id, "Something go wrong...", reply_markup=keyboard)

bot.polling(none_stop = True, interval = 0)